# Chapter 31: Advanced Design Patterns

Design patterns are proven solutions to common software design problems. In TypeScript, these patterns gain additional power through static typing, enabling compile-time verification of pattern implementations and better IDE support. This chapter explores essential design patterns with type-safe implementations following industry best practices.

---

## 31.1 Singleton Pattern

The Singleton pattern ensures a class has only one instance and provides a global point of access to it. This is useful for managing shared resources like database connections, configuration settings, or caching services.

### Implementation

```typescript
// Classic Singleton with private constructor
class DatabaseConnection {
  // Static instance property
  private static instance: DatabaseConnection | null = null;
  
  // Private constructor prevents direct instantiation
  private constructor(
    private connectionString: string,
    private isConnected: boolean = false
  ) {
    console.log(`Creating connection to: ${connectionString}`);
  }
  
  // Public static method to get the instance
  public static getInstance(connectionString: string = "default"): DatabaseConnection {
    if (DatabaseConnection.instance === null) {
      DatabaseConnection.instance = new DatabaseConnection(connectionString);
    }
    return DatabaseConnection.instance;
  }
  
  // Instance methods
  public connect(): void {
    if (!this.isConnected) {
      this.isConnected = true;
      console.log("Connected to database");
    }
  }
  
  public disconnect(): void {
    if (this.isConnected) {
      this.isConnected = false;
      console.log("Disconnected from database");
    }
  }
  
  public query(sql: string): unknown[] {
    if (!this.isConnected) {
      throw new Error("Not connected to database");
    }
    console.log(`Executing: ${sql}`);
    return [];
  }
}

// Usage
const db1 = DatabaseConnection.getInstance("postgres://localhost");
const db2 = DatabaseConnection.getInstance("mysql://localhost");

console.log(db1 === db2); // true - same instance

db1.connect();
db1.query("SELECT * FROM users");
db1.disconnect();
```

**Explanation:**
- The `private constructor` prevents creating instances with `new DatabaseConnection()`
- `static instance` holds the single instance
- `getInstance()` provides controlled access, creating the instance only on first call
- All subsequent calls return the same instance, ensuring singleton behavior

### Thread-Safe Singleton (for async environments)

```typescript
// Singleton for async initialization
class AsyncConfigManager {
  private static instance: AsyncConfigManager | null = null;
  private config: Record<string, unknown> | null = null;
  private initialized = false;
  
  private constructor() {}
  
  static async getInstance(): Promise<AsyncConfigManager> {
    if (AsyncConfigManager.instance === null) {
      AsyncConfigManager.instance = new AsyncConfigManager();
      await AsyncConfigManager.instance.initialize();
    }
    return AsyncConfigManager.instance;
  }
  
  private async initialize(): Promise<void> {
    if (this.initialized) return;
    
    // Simulate async config loading
    this.config = await fetch("/api/config").then(r => r.json());
    this.initialized = true;
  }
  
  get<T>(key: string, defaultValue?: T): T {
    if (!this.initialized) {
      throw new Error("ConfigManager not initialized");
    }
    return (this.config?.[key] as T) ?? defaultValue!;
  }
}

// Usage
async function main() {
  const config = await AsyncConfigManager.getInstance();
  const apiUrl = config.get<string>("apiUrl");
  const timeout = config.get<number>("timeout", 5000);
}

main();
```

---

## 31.2 Factory Pattern

The Factory pattern provides an interface for creating objects without specifying the exact class to create. It delegates instantiation to subclasses or factory methods, promoting loose coupling.

### Simple Factory

```typescript
// Product interface
interface Notification {
  send(message: string, recipient: string): void;
  getChannel(): string;
}

// Concrete products
class EmailNotification implements Notification {
  send(message: string, recipient: string): void {
    console.log(`[EMAIL] To: ${recipient}`);
    console.log(`Message: ${message}`);
  }
  
  getChannel(): string {
    return "email";
  }
}

class SMSNotification implements Notification {
  send(message: string, recipient: string): void {
    console.log(`[SMS] To: ${recipient}`);
    console.log(`Message: ${message.substring(0, 160)}`); // SMS limit
  }
  
  getChannel(): string {
    return "sms";
  }
}

class PushNotification implements Notification {
  send(message: string, recipient: string): void {
    console.log(`[PUSH] To device: ${recipient}`);
    console.log(`Message: ${message}`);
    console.log(`Badge: 1`);
  }
  
  getChannel(): string {
    return "push";
  }
}

// Factory function
type NotificationChannel = "email" | "sms" | "push";

function createNotification(channel: NotificationChannel): Notification {
  switch (channel) {
    case "email":
      return new EmailNotification();
    case "sms":
      return new SMSNotification();
    case "push":
      return new PushNotification();
    default:
      // Exhaustive check
      const _exhaustive: never = channel;
      throw new Error(`Unknown channel: ${_exhaustive}`);
  }
}

// Usage
const emailNotifier = createNotification("email");
emailNotifier.send("Welcome!", "user@example.com");

const smsNotifier = createNotification("sms");
smsNotifier.send("Your code is 123456", "+1234567890");
```

**Explanation:**
- The `Notification` interface defines the contract all products must implement
- Concrete classes (`EmailNotification`, `SMSNotification`, etc.) implement the interface
- `createNotification` factory function encapsulates creation logic
- Adding new notification types only requires adding a new case to the factory

### Abstract Factory

```typescript
// Abstract product interfaces
interface Button {
  render(): void;
  onClick(handler: () => void): void;
}

interface TextInput {
  render(): void;
  getValue(): string;
  setValue(value: string): void;
}

interface Checkbox {
  render(): void;
  isChecked(): boolean;
  setChecked(checked: boolean): void;
}

// Abstract factory interface
interface UIFactory {
  createButton(label: string): Button;
  createTextInput(placeholder: string): TextInput;
  createCheckbox(label: string): Checkbox;
}

// Concrete products: Material Design
class MaterialButton implements Button {
  constructor(private label: string) {}
  
  render(): void {
    console.log(`[Material] Rendering button: ${this.label}`);
  }
  
  onClick(handler: () => void): void {
    console.log(`[Material] Attaching click handler to ${this.label}`);
  }
}

class MaterialTextInput implements TextInput {
  private value = "";
  
  constructor(private placeholder: string) {}
  
  render(): void {
    console.log(`[Material] Rendering input with placeholder: ${this.placeholder}`);
  }
  
  getValue(): string {
    return this.value;
  }
  
  setValue(value: string): void {
    this.value = value;
  }
}

class MaterialCheckbox implements Checkbox {
  private checked = false;
  
  constructor(private label: string) {}
  
  render(): void {
    console.log(`[Material] Rendering checkbox: ${this.label}`);
  }
  
  isChecked(): boolean {
    return this.checked;
  }
  
  setChecked(checked: boolean): void {
    this.checked = checked;
  }
}

// Concrete factory: Material Design
class MaterialUIFactory implements UIFactory {
  createButton(label: string): Button {
    return new MaterialButton(label);
  }
  
  createTextInput(placeholder: string): TextInput {
    return new MaterialTextInput(placeholder);
  }
  
  createCheckbox(label: string): Checkbox {
    return new MaterialCheckbox(label);
  }
}

// Concrete products: Bootstrap (simplified)
class BootstrapButton implements Button {
  constructor(private label: string) {}
  
  render(): void {
    console.log(`[Bootstrap] Rendering btn-primary: ${this.label}`);
  }
  
  onClick(handler: () => void): void {
    console.log(`[Bootstrap] Adding onclick to ${this.label}`);
  }
}

// Concrete factory: Bootstrap
class BootstrapUIFactory implements UIFactory {
  createButton(label: string): Button {
    return new BootstrapButton(label);
  }
  
  createTextInput(placeholder: string): TextInput {
    // Implementation...
    return {} as TextInput;
  }
  
  createCheckbox(label: string): Checkbox {
    // Implementation...
    return {} as Checkbox;
  }
}

// Client code - works with any factory
function createLoginForm(factory: UIFactory): void {
  const usernameInput = factory.createTextInput("Enter username");
  const passwordInput = factory.createTextInput("Enter password");
  const rememberCheckbox = factory.createCheckbox("Remember me");
  const submitButton = factory.createButton("Login");
  
  usernameInput.render();
  passwordInput.render();
  rememberCheckbox.render();
  submitButton.render();
}

// Usage
console.log("=== Material Design ===");
createLoginForm(new MaterialUIFactory());

console.log("\n=== Bootstrap ===");
createLoginForm(new BootstrapUIFactory());
```

---

## 31.3 Builder Pattern

The Builder pattern separates the construction of a complex object from its representation, allowing the same construction process to create different representations. It's ideal for objects with many optional parameters or complex construction logic.

### Implementation

```typescript
// Product: Complex object being built
class HttpRequest {
  constructor(
    public method: "GET" | "POST" | "PUT" | "DELETE",
    public url: string,
    public headers: Record<string, string>,
    public body: unknown,
    public timeout: number,
    public retries: number,
    public cache: boolean
  ) {}
  
  toString(): string {
    return `${this.method} ${this.url}`;
  }
}

// Builder class
class HttpRequestBuilder {
  private method: "GET" | "POST" | "PUT" | "DELETE" = "GET";
  private url = "";
  private headers: Record<string, string> = {};
  private body: unknown = null;
  private timeout = 5000;
  private retries = 0;
  private cache = false;
  
  // Fluent API methods return 'this' for chaining
  setMethod(method: "GET" | "POST" | "PUT" | "DELETE"): this {
    this.method = method;
    return this;
  }
  
  setUrl(url: string): this {
    this.url = url;
    return this;
  }
  
  addHeader(key: string, value: string): this {
    this.headers[key] = value;
    return this;
  }
  
  setBody(body: unknown): this {
    this.body = body;
    return this;
  }
  
  setTimeout(timeout: number): this {
    this.timeout = timeout;
    return this;
  }
  
  setRetries(retries: number): this {
    this.retries = retries;
    return this;
  }
  
  enableCache(): this {
    this.cache = true;
    return this;
  }
  
  // Build method returns the final product
  build(): HttpRequest {
    if (!this.url) {
      throw new Error("URL is required");
    }
    
    return new HttpRequest(
      this.method,
      this.url,
      this.headers,
      this.body,
      this.timeout,
      this.retries,
      this.cache
    );
  }
}

// Usage - Fluent API
const request = new HttpRequestBuilder()
  .setMethod("POST")
  .setUrl("https://api.example.com/users")
  .addHeader("Content-Type", "application/json")
  .addHeader("Authorization", "Bearer token123")
  .setBody({ name: "John", email: "john@example.com" })
  .setTimeout(10000)
  .setRetries(3)
  .build();

console.log(request.toString()); // POST https://api.example.com/users

// Step-by-step building
const builder = new HttpRequestBuilder();
builder.setMethod("GET");
builder.setUrl("https://api.example.com/status");
builder.setTimeout(3000);
const statusRequest = builder.build();
```

**Explanation:**
- The `HttpRequest` class is the product with many properties
- `HttpRequestBuilder` provides a fluent API for step-by-step construction
- Each builder method returns `this` for method chaining
- The `build()` method validates and creates the final object
- This pattern is especially useful when objects have many optional parameters

### Step Builder Pattern

For complex objects requiring specific construction steps:

```typescript
// Step Builder for a complex configuration
interface DatabaseConfig {
  host: string;
  port: number;
  username: string;
  password: string;
  database: string;
  ssl: boolean;
  poolSize: number;
  timeout: number;
}

// Step interfaces enforce construction order
interface HostStep {
  host(host: string): PortStep;
}

interface PortStep {
  port(port: number): CredentialsStep;
}

interface CredentialsStep {
  username(username: string): PasswordStep;
}

interface PasswordStep {
  password(password: string): DatabaseStep;
}

interface DatabaseStep {
  database(name: string): OptionalStep;
}

interface OptionalStep {
  ssl(enabled: boolean): OptionalStep;
  poolSize(size: number): OptionalStep;
  timeout(ms: number): OptionalStep;
  build(): DatabaseConfig;
}

class DatabaseConfigBuilder implements HostStep, PortStep, CredentialsStep, PasswordStep, DatabaseStep, OptionalStep {
  private config: Partial<DatabaseConfig> = {};
  
  host(host: string): PortStep {
    this.config.host = host;
    return this;
  }
  
  port(port: number): CredentialsStep {
    this.config.port = port;
    return this;
  }
  
  username(username: string): PasswordStep {
    this.config.username = username;
    return this;
  }
  
  password(password: string): DatabaseStep {
    this.config.password = password;
    return this;
  }
  
  database(name: string): OptionalStep {
    this.config.database = name;
    return this;
  }
  
  ssl(enabled: boolean): OptionalStep {
    this.config.ssl = enabled;
    return this;
  }
  
  poolSize(size: number): OptionalStep {
    this.config.poolSize = size;
    return this;
  }
  
  timeout(ms: number): OptionalStep {
    this.config.timeout = ms;
    return this;
  }
  
  build(): DatabaseConfig {
    if (!this.config.host || !this.config.port || !this.config.username || 
        !this.config.password || !this.config.database) {
      throw new Error("Missing required configuration");
    }
    
    return {
      host: this.config.host,
      port: this.config.port,
      username: this.config.username,
      password: this.config.password,
      database: this.config.database,
      ssl: this.config.ssl ?? false,
      poolSize: this.config.poolSize ?? 10,
      timeout: this.config.timeout ?? 30000
    };
  }
}

// Usage - enforced order
const config = new DatabaseConfigBuilder()
  .host("localhost")
  .port(5432)
  .username("admin")
  .password("secret")
  .database("myapp")
  .ssl(true)
  .poolSize(20)
  .timeout(60000)
  .build();

console.log(config);
// TypeScript enforces: host -> port -> username -> password -> database -> optional
```

---

## 31.4 Observer Pattern

The Observer pattern defines a one-to-many dependency between objects. When one object changes state, all its dependents are notified and updated automatically. This is fundamental for event-driven programming.

### Implementation

```typescript
// Observer interface
interface Observer<T> {
  update(data: T): void;
}

// Subject interface
interface Subject<T> {
  attach(observer: Observer<T>): void;
  detach(observer: Observer<T>): void;
  notify(data: T): void;
}

// Concrete Subject: Weather Station
class WeatherStation implements Subject<WeatherData> {
  private observers: Set<Observer<WeatherData>> = new Set();
  private currentData: WeatherData | null = null;
  
  attach(observer: Observer<WeatherData>): void {
    this.observers.add(observer);
    console.log("Observer attached");
    
    // Immediately notify with current data if available
    if (this.currentData) {
      observer.update(this.currentData);
    }
  }
  
  detach(observer: Observer<WeatherData>): void {
    this.observers.delete(observer);
    console.log("Observer detached");
  }
  
  notify(data: WeatherData): void {
    this.currentData = data;
    this.observers.forEach(observer => observer.update(data));
  }
  
  // Business logic
  setMeasurements(temperature: number, humidity: number, pressure: number): void {
    const data: WeatherData = {
      temperature,
      humidity,
      pressure,
      timestamp: new Date()
    };
    this.notify(data);
  }
}

interface WeatherData {
  temperature: number;
  humidity: number;
  pressure: number;
  timestamp: Date;
}

// Concrete Observers
class CurrentConditionsDisplay implements Observer<WeatherData> {
  constructor(private station: WeatherStation) {
    this.station.attach(this);
  }
  
  update(data: WeatherData): void {
    console.log(`
      Current Conditions:
      Temperature: ${data.temperature}°F
      Humidity: ${data.humidity}%
      Time: ${data.timestamp.toLocaleTimeString()}
    `);
  }
  
  unsubscribe(): void {
    this.station.detach(this);
  }
}

class StatisticsDisplay implements Observer<WeatherData> {
  private temperatures: number[] = [];
  
  constructor(private station: WeatherStation) {
    this.station.attach(this);
  }
  
  update(data: WeatherData): void {
    this.temperatures.push(data.temperature);
    
    const avg = this.temperatures.reduce((a, b) => a + b, 0) / this.temperatures.length;
    const max = Math.max(...this.temperatures);
    const min = Math.min(...this.temperatures);
    
    console.log(`
      Statistics:
      Avg: ${avg.toFixed(1)}°F
      Max: ${max}°F
      Min: ${min}°F
      Readings: ${this.temperatures.length}
    `);
  }
}

// Usage
const weatherStation = new WeatherStation();

const currentDisplay = new CurrentConditionsDisplay(weatherStation);
const statsDisplay = new StatisticsDisplay(weatherStation);

weatherStation.setMeasurements(80, 65, 30.4);
weatherStation.setMeasurements(82, 70, 29.2);
weatherStation.setMeasurements(78, 90, 29.2);

// Unsubscribe
currentDisplay.unsubscribe();
```

**Explanation:**
- **Subject** (`WeatherStation`): Maintains a list of observers and notifies them of state changes
- **Observer** interface: Defines the contract for objects that should be notified
- **Concrete Observers**: Implement specific reactions to state changes
- **Loose coupling**: Subjects and observers know only about interfaces, not concrete implementations

---

## 31.5 Strategy Pattern

The Strategy pattern defines a family of algorithms, encapsulates each one, and makes them interchangeable. It lets the algorithm vary independently from clients that use it.

### Implementation

```typescript
// Strategy interface
interface PaymentStrategy {
  pay(amount: number): Promise<PaymentResult>;
  validate(): boolean;
  getName(): string;
}

interface PaymentResult {
  success: boolean;
  transactionId?: string;
  message: string;
  timestamp: Date;
}

// Concrete Strategy: Credit Card
class CreditCardStrategy implements PaymentStrategy {
  constructor(
    private cardNumber: string,
    private cvv: string,
    private expiryDate: string,
    private cardHolder: string
  ) {}
  
  validate(): boolean {
    // Luhn algorithm validation
    const clean = this.cardNumber.replace(/\s/g, "");
    if (!/^\d{13,19}$/.test(clean)) return false;
    
    let sum = 0;
    let isEven = false;
    
    for (let i = clean.length - 1; i >= 0; i--) {
      let digit = parseInt(clean.charAt(i), 10);
      
      if (isEven) {
        digit *= 2;
        if (digit > 9) digit -= 9;
      }
      
      sum += digit;
      isEven = !isEven;
    }
    
    return sum % 10 === 0;
  }
  
  async pay(amount: number): Promise<PaymentResult> {
    if (!this.validate()) {
      return {
        success: false,
        message: "Invalid credit card",
        timestamp: new Date()
      };
    }
    
    // Simulate API call
    await new Promise(resolve => setTimeout(resolve, 1000));
    
    return {
      success: true,
      transactionId: `CC-${Date.now()}`,
      message: `Paid $${amount} via Credit Card`,
      timestamp: new Date()
    };
  }
  
  getName(): string {
    return "Credit Card";
  }
}

// Concrete Strategy: PayPal
class PayPalStrategy implements PaymentStrategy {
  constructor(
    private email: string,
    private password: string
  ) {}
  
  validate(): boolean {
    return /^[^\s@]+@[^\s@]+\.[^\s@]+$/.test(this.email) && 
           this.password.length >= 8;
  }
  
  async pay(amount: number): Promise<PaymentResult> {
    if (!this.validate()) {
      return {
        success: false,
        message: "Invalid PayPal credentials",
        timestamp: new Date()
      };
    }
    
    await new Promise(resolve => setTimeout(resolve, 800));
    
    return {
      success: true,
      transactionId: `PP-${Date.now()}`,
      message: `Paid $${amount} via PayPal`,
      timestamp: new Date()
    };
  }
  
  getName(): string {
    return "PayPal";
  }
}

// Concrete Strategy: Cryptocurrency
class CryptoStrategy implements PaymentStrategy {
  constructor(
    private walletAddress: string,
    private privateKey: string,
    private currency: "BTC" | "ETH" | "SOL"
  ) {}
  
  validate(): boolean {
    // Basic validation for wallet addresses
    const patterns = {
      BTC: /^[13][a-km-zA-HJ-NP-Z1-9]{25,34}$/,
      ETH: /^0x[a-fA-F0-9]{40}$/,
      SOL: /^[1-9A-HJ-NP-Za-km-z]{32,44}$/
    };
    return patterns[this.currency].test(this.walletAddress);
  }
  
  async pay(amount: number): Promise<PaymentResult> {
    if (!this.validate()) {
      return {
        success: false,
        message: "Invalid wallet address",
        timestamp: new Date()
      };
    }
    
    await new Promise(resolve => setTimeout(resolve, 2000));
    
    return {
      success: true,
      transactionId: `${this.currency}-${Date.now()}`,
      message: `Paid ${amount} ${this.currency}`,
      timestamp: new Date()
    };
  }
  
  getName(): string {
    return `${this.currency} Wallet`;
  }
}

// Context: Shopping Cart
class ShoppingCart {
  private items: { name: string; price: number }[] = [];
  private paymentStrategy: PaymentStrategy | null = null;
  
  addItem(name: string, price: number): void {
    this.items.push({ name, price });
  }
  
  getTotal(): number {
    return this.items.reduce((sum, item) => sum + item.price, 0);
  }
  
  setPaymentStrategy(strategy: PaymentStrategy): void {
    this.paymentStrategy = strategy;
  }
  
  async checkout(): Promise<PaymentResult> {
    if (!this.paymentStrategy) {
      return {
        success: false,
        message: "No payment method selected",
        timestamp: new Date()
      };
    }
    
    const total = this.getTotal();
    console.log(`\nChecking out ${this.items.length} items, total: $${total}`);
    console.log(`Using: ${this.paymentStrategy.getName()}`);
    
    return this.paymentStrategy.pay(total);
  }
}

// Usage
async function main() {
  const cart = new ShoppingCart();
  cart.addItem("Laptop", 999.99);
  cart.addItem("Mouse", 29.99);
  cart.addItem("Keyboard", 79.99);
  
  // Pay with credit card
  cart.setPaymentStrategy(
    new CreditCardStrategy("4532015112830366", "123", "12/25", "John Doe")
  );
  const result1 = await cart.checkout();
  console.log(result1);
  
  // Pay with PayPal
  cart.setPaymentStrategy(
    new PayPalStrategy("john@example.com", "securePassword123")
  );
  const result2 = await cart.checkout();
  console.log(result2);
  
  // Pay with Crypto
  cart.setPaymentStrategy(
    new CryptoStrategy("0x742d35Cc6634C0532925a3b844Bc9e7595f0bEb", "key", "ETH")
  );
  const result3 = await cart.checkout();
  console.log(result3);
}

main();
```

**Explanation:**
- **Strategy Interface**: Defines the contract for all payment algorithms
- **Concrete Strategies**: Implement specific payment methods (Credit Card, PayPal, Crypto)
- **Context** (`ShoppingCart`): Uses a strategy without knowing its concrete implementation
- **Runtime Strategy Switching**: Can change payment method at runtime

---

## 31.6 Command Pattern

The Command pattern encapsulates a request as an object, thereby letting you parameterize clients with different requests, queue or log requests, and support undoable operations.

### Implementation

```typescript
// Command interface
interface Command {
  execute(): void;
  undo(): void;
  getDescription(): string;
}

// Receiver: The object that performs the actual work
class TextEditor {
  private content = "";
  private clipboard = "";
  
  getContent(): string {
    return this.content;
  }
  
  setContent(content: string): void {
    this.content = content;
  }
  
  getClipboard(): string {
    return this.clipboard;
  }
  
  setClipboard(text: string): void {
    this.clipboard = text;
  }
  
  insertText(position: number, text: string): void {
    this.content = this.content.slice(0, position) + text + this.content.slice(position);
  }
  
  deleteText(start: number, end: number): string {
    const deleted = this.content.slice(start, end);
    this.content = this.content.slice(0, start) + this.content.slice(end);
    return deleted;
  }
}

// Concrete Commands
class WriteCommand implements Command {
  private previousContent: string;
  
  constructor(
    private editor: TextEditor,
    private text: string
  ) {
    this.previousContent = editor.getContent();
  }
  
  execute(): void {
    this.editor.setContent(this.previousContent + this.text);
  }
  
  undo(): void {
    this.editor.setContent(this.previousContent);
  }
  
  getDescription(): string {
    return `Write "${this.text}"`;
  }
}

class DeleteCommand implements Command {
  private deletedText: string;
  
  constructor(
    private editor: TextEditor,
    private start: number,
    private end: number
  ) {}
  
  execute(): void {
    this.deletedText = this.editor.deleteText(this.start, this.end);
  }
  
  undo(): void {
    this.editor.insertText(this.start, this.deletedText);
  }
  
  getDescription(): string {
    return `Delete from ${this.start} to ${this.end}`;
  }
}

class CopyCommand implements Command {
  constructor(
    private editor: TextEditor,
    private start: number,
    private end: number
  ) {}
  
  execute(): void {
    const text = this.editor.getContent().slice(this.start, this.end);
    this.editor.setClipboard(text);
  }
  
  undo(): void {
    // Copy is not undoable in this implementation
    this.editor.setClipboard("");
  }
  
  getDescription(): string {
    return `Copy from ${this.start} to ${this.end}`;
  }
}

// Invoker: Manages command execution and history
class CommandManager {
  private history: Command[] = [];
  private redoStack: Command[] = [];
  
  execute(command: Command): void {
    command.execute();
    this.history.push(command);
    this.redoStack = []; // Clear redo stack on new action
    console.log(`Executed: ${command.getDescription()}`);
  }
  
  undo(): void {
    const command = this.history.pop();
    if (command) {
      command.undo();
      this.redoStack.push(command);
      console.log(`Undone: ${command.getDescription()}`);
    } else {
      console.log("Nothing to undo");
    }
  }
  
  redo(): void {
    const command = this.redoStack.pop();
    if (command) {
      command.execute();
      this.history.push(command);
      console.log(`Redone: ${command.getDescription()}`);
    } else {
      console.log("Nothing to redo");
    }
  }
  
  getHistory(): string[] {
    return this.history.map(cmd => cmd.getDescription());
  }
}

// Usage
const editor = new TextEditor();
const commandManager = new CommandManager();

console.log("=== Text Editor Demo ===\n");

// Write some text
commandManager.execute(new WriteCommand(editor, "Hello World"));
console.log("Content:", editor.getContent());

// Delete part of it
commandManager.execute(new DeleteCommand(editor, 5, 11));
console.log("Content:", editor.getContent());

// Copy text
commandManager.execute(new CopyCommand(editor, 0, 5));
console.log("Clipboard:", editor.getClipboard());

// Undo operations
console.log("\n--- Undoing ---");
commandManager.undo(); // Undo copy
commandManager.undo(); // Undo delete
console.log("Content:", editor.getContent());

// Redo
console.log("\n--- Redoing ---");
commandManager.redo();
console.log("Content:", editor.getContent());

console.log("\nHistory:", commandManager.getHistory());
```

**Explanation:**
- **Command Interface**: Defines execute, undo, and description methods
- **Receiver** (`TextEditor`): The actual object being manipulated
- **Concrete Commands**: Encapsulate specific operations (Write, Delete, Copy)
- **Invoker** (`CommandManager`): Executes commands and maintains history for undo/redo
- **Benefits**: Operations become objects that can be queued, logged, undone, and serialized

---

## 31.7 Decorator Pattern

The Decorator pattern allows behavior to be added to individual objects, either statically or dynamically, without affecting the behavior of other objects from the same class. It's a flexible alternative to subclassing.

### Implementation

```typescript
// Component interface
interface Coffee {
  cost(): number;
  description(): string;
  ingredients(): string[];
}

// Concrete Component: Simple Coffee
class SimpleCoffee implements Coffee {
  cost(): number {
    return 10;
  }
  
  description(): string {
    return "Simple coffee";
  }
  
  ingredients(): string[] {
    return ["coffee", "water"];
  }
}

// Base Decorator
abstract class CoffeeDecorator implements Coffee {
  protected decoratedCoffee: Coffee;
  
  constructor(coffee: Coffee) {
    this.decoratedCoffee = coffee;
  }
  
  cost(): number {
    return this.decoratedCoffee.cost();
  }
  
  description(): string {
    return this.decoratedCoffee.description();
  }
  
  ingredients(): string[] {
    return this.decoratedCoffee.ingredients();
  }
}

// Concrete Decorators
class MilkDecorator extends CoffeeDecorator {
  cost(): number {
    return super.cost() + 2;
  }
  
  description(): string {
    return `${super.description()}, milk`;
  }
  
  ingredients(): string[] {
    return [...super.ingredients(), "milk"];
  }
}

class SugarDecorator extends CoffeeDecorator {
  private sugarLevel: number;
  
  constructor(coffee: Coffee, sugarLevel: number = 1) {
    super(coffee);
    this.sugarLevel = sugarLevel;
  }
  
  cost(): number {
    return super.cost() + (0.5 * this.sugarLevel);
  }
  
  description(): string {
    return `${super.description()}, ${this.sugarLevel}x sugar`;
  }
  
  ingredients(): string[] {
    return [...super.ingredients(), `sugar (${this.sugarLevel})`];
  }
}

class WhipDecorator extends CoffeeDecorator {
  cost(): number {
    return super.cost() + 3;
  }
  
  description(): string {
    return `${super.description()}, whipped cream`;
  }
  
  ingredients(): string[] {
    return [...super.ingredients(), "whipped cream"];
  }
}

// Usage - Composing decorators
console.log("=== Coffee Shop ===\n");

// Simple coffee
let coffee: Coffee = new SimpleCoffee();
console.log(`Order: ${coffee.description()}`);
console.log(`Cost: $${coffee.cost()}`);
console.log(`Ingredients: ${coffee.ingredients().join(", ")}`);
console.log();

// Add milk
coffee = new MilkDecorator(coffee);
console.log(`Order: ${coffee.description()}`);
console.log(`Cost: $${coffee.cost()}`);
console.log();

// Add sugar
coffee = new SugarDecorator(coffee, 2);
console.log(`Order: ${coffee.description()}`);
console.log(`Cost: $${coffee.cost()}`);
console.log();

// Add whipped cream
coffee = new WhipDecorator(coffee);
console.log(`Order: ${coffee.description()}`);
console.log(`Cost: $${coffee.cost()}`);
console.log(`Ingredients: ${coffee.ingredients().join(", ")}`);

// Alternative: Create specific combinations
function createLatte(): Coffee {
  return new MilkDecorator(new SimpleCoffee());
}

function createMocha(): Coffee {
  return new WhipDecorator(
    new SugarDecorator(
      new MilkDecorator(new SimpleCoffee()),
      1
    )
  );
}

console.log("\n=== Predefined Orders ===");
const latte = createLatte();
console.log(`Latte: ${latte.description()} - $${latte.cost()}`);

const mocha = createMocha();
console.log(`Mocha: ${mocha.description()} - $${mocha.cost()}`);
```

**Explanation:**
- **Component Interface** (`Coffee`): Defines the interface for objects that can have responsibilities added
- **Concrete Component** (`SimpleCoffee`): The base object being decorated
- **Decorator Base Class**: Maintains a reference to the component and implements the same interface
- **Concrete Decorators**: Add specific behaviors (Milk, Sugar, Whip) by extending the decorator
- **Composition**: Decorators can wrap other decorators, creating flexible combinations

---

## 31.8 Proxy Pattern

The Proxy pattern provides a surrogate or placeholder for another object to control access to it. Proxies can add logging, validation, caching, lazy loading, or access control without changing the original object.

### Implementation

```typescript
// Subject interface
interface UserService {
  getUser(id: number): Promise<User>;
  saveUser(user: User): Promise<void>;
  deleteUser(id: number): Promise<void>;
  listUsers(): Promise<User[]>;
}

interface User {
  id: number;
  name: string;
  email: string;
  role: "admin" | "user" | "guest";
}

// Real Subject
class RealUserService implements UserService {
  private users = new Map<number, User>();
  private nextId = 1;
  
  async getUser(id: number): Promise<User> {
    await this.simulateNetwork();
    const user = this.users.get(id);
    if (!user) {
      throw new Error(`User ${id} not found`);
    }
    return user;
  }
  
  async saveUser(user: User): Promise<void> {
    await this.simulateNetwork();
    if (user.id === 0) {
      user.id = this.nextId++;
    }
    this.users.set(user.id, user);
  }
  
  async deleteUser(id: number): Promise<void> {
    await this.simulateNetwork();
    this.users.delete(id);
  }
  
  async listUsers(): Promise<User[]> {
    await this.simulateNetwork();
    return Array.from(this.users.values());
  }
  
  private simulateNetwork(): Promise<void> {
    return new Promise(resolve => setTimeout(resolve, 100));
  }
}

// Proxy: Logging Proxy
class LoggingProxy implements UserService {
  constructor(private service: UserService) {}
  
  async getUser(id: number): Promise<User> {
    console.log(`[LOG] Getting user ${id}`);
    const start = Date.now();
    try {
      const user = await this.service.getUser(id);
      console.log(`[LOG] Got user ${id} in ${Date.now() - start}ms`);
      return user;
    } catch (error) {
      console.error(`[LOG] Failed to get user ${id}: ${error}`);
      throw error;
    }
  }
  
  async saveUser(user: User): Promise<void> {
    console.log(`[LOG] Saving user ${user.id || "new"}`);
    return this.service.saveUser(user);
  }
  
  async deleteUser(id: number): Promise<void> {
    console.log(`[LOG] Deleting user ${id}`);
    return this.service.deleteUser(id);
  }
  
  async listUsers(): Promise<User[]> {
    console.log(`[LOG] Listing users`);
    return this.service.listUsers();
  }
}

// Proxy: Caching Proxy
class CachingProxy implements UserService {
  private cache = new Map<string, { data: unknown; timestamp: number }>();
  private ttl: number; // Time to live in milliseconds
  
  constructor(private service: UserService, ttlSeconds: number = 60) {
    this.ttl = ttlSeconds * 1000;
  }
  
  async getUser(id: number): Promise<User> {
    const cacheKey = `user_${id}`;
    const cached = this.getFromCache<User>(cacheKey);
    
    if (cached) {
      console.log(`[CACHE] Hit for user ${id}`);
      return cached;
    }
    
    console.log(`[CACHE] Miss for user ${id}`);
    const user = await this.service.getUser(id);
    this.setCache(cacheKey, user);
    return user;
  }
  
  async saveUser(user: User): Promise<void> {
    await this.service.saveUser(user);
    // Invalidate cache for this user
    this.cache.delete(`user_${user.id}`);
    // Also invalidate list cache
    this.cache.delete("users_list");
  }
  
  async deleteUser(id: number): Promise<void> {
    await this.service.deleteUser(id);
    this.cache.delete(`user_${id}`);
    this.cache.delete("users_list");
  }
  
  async listUsers(): Promise<User[]> {
    const cacheKey = "users_list";
    const cached = this.getFromCache<User[]>(cacheKey);
    
    if (cached) {
      console.log(`[CACHE] Hit for user list`);
      return cached;
    }
    
    console.log(`[CACHE] Miss for user list`);
    const users = await this.service.listUsers();
    this.setCache(cacheKey, users);
    return users;
  }
  
  private getFromCache<T>(key: string): T | null {
    const entry = this.cache.get(key);
    if (!entry) return null;
    
    if (Date.now() - entry.timestamp > this.ttl) {
      this.cache.delete(key);
      return null;
    }
    
    return entry.data as T;
  }
  
  private setCache(key: string, data: unknown): void {
    this.cache.set(key, { data, timestamp: Date.now() });
  }
  
  clearCache(): void {
    this.cache.clear();
    console.log("[CACHE] Cleared");
  }
}

// Proxy: Access Control Proxy
class AuthorizationProxy implements UserService {
  constructor(
    private service: UserService,
    private currentUserRole: "admin" | "user" | "guest"
  ) {}
  
  async getUser(id: number): Promise<User> {
    // Everyone can read
    return this.service.getUser(id);
  }
  
  async saveUser(user: User): Promise<void> {
    this.checkPermission("admin", "save users");
    return this.service.saveUser(user);
  }
  
  async deleteUser(id: number): Promise<void> {
    this.checkPermission("admin", "delete users");
    return this.service.deleteUser(id);
  }
  
  async listUsers(): Promise<User[]> {
    this.checkPermission("user", "list users");
    return this.service.listUsers();
  }
  
  private checkPermission(required: "admin" | "user" | "guest", action: string): void {
    const hierarchy = { guest: 0, user: 1, admin: 2 };
    if (hierarchy[this.currentUserRole] < hierarchy[required]) {
      throw new Error(`Access denied: ${this.currentUserRole} cannot ${action}`);
    }
  }
}

// Usage: Composing proxies
async function demonstrateProxies() {
  console.log("=== Proxy Pattern Demo ===\n");
  
  // Base service
  const realService = new RealUserService();
  
  // Add logging
  const loggedService = new LoggingProxy(realService);
  
  // Add caching (wraps logged service)
  const cachedService = new CachingProxy(loggedService, 5); // 5 second TTL
  
  // Add authorization (outermost layer)
  const authorizedService = new AuthorizationProxy(cachedService, "admin");
  
  // Operations go through all layers: Auth -> Cache -> Log -> Real
  console.log("--- Creating User ---");
  await authorizedService.saveUser({
    id: 0,
    name: "John Doe",
    email: "john@example.com",
    role: "user"
  });
  
  console.log("\n--- Fetching User (Cache Miss) ---");
  const user1 = await authorizedService.getUser(1);
  
  console.log("\n--- Fetching User (Cache Hit) ---");
  const user2 = await authorizedService.getUser(1); // From cache
  
  console.log("\n--- Listing Users ---");
  await authorizedService.listUsers();
  
  // Test authorization
  console.log("\n--- Testing Authorization ---");
  const guestService = new AuthorizationProxy(realService, "guest");
  try {
    await guestService.deleteUser(1);
  } catch (error) {
    console.error(error);
  }
}

demonstrateProxies();
```

**Explanation:**
- **Logging Proxy**: Adds logging before and after operations without changing the service
- **Caching Proxy**: Adds caching layer with TTL, invalidating on writes
- **Authorization Proxy**: Adds access control checks before operations
- **Composition**: Proxies can be chained/wrapped to combine behaviors
- **Single Responsibility**: Each proxy handles one aspect (logging, caching, auth)

---

## 31.3 Builder Pattern

The Builder pattern separates the construction of a complex object from its representation, allowing the same construction process to create different representations. It's particularly useful for objects with many optional parameters.

### Implementation

```typescript
// Product: Complex object
class HttpRequest {
  constructor(
    public method: "GET" | "POST" | "PUT" | "DELETE",
    public url: string,
    public headers: Record<string, string>,
    public body: unknown,
    public timeout: number,
    public retries: number,
    public cache: boolean,
    public authToken?: string
  ) {}
  
  toString(): string {
    return `${this.method} ${this.url}`;
  }
}

// Builder interface
interface RequestBuilder {
  setMethod(method: "GET" | "POST" | "PUT" | "DELETE"): this;
  setUrl(url: string): this;
  addHeader(key: string, value: string): this;
  setBody(body: unknown): this;
  setTimeout(timeout: number): this;
  setRetries(retries: number): this;
  enableCache(): this;
  setAuthToken(token: string): this;
  build(): HttpRequest;
}

// Concrete Builder
class HttpRequestBuilder implements RequestBuilder {
  private method: "GET" | "POST" | "PUT" | "DELETE" = "GET";
  private url = "";
  private headers: Record<string, string> = {};
  private body: unknown = null;
  private timeout = 5000;
  private retries = 0;
  private cache = false;
  private authToken?: string;
  
  setMethod(method: "GET" | "POST" | "PUT" | "DELETE"): this {
    this.method = method;
    return this;
  }
  
  setUrl(url: string): this {
    this.url = url;
    return this;
  }
  
  addHeader(key: string, value: string): this {
    this.headers[key] = value;
    return this;
  }
  
  setBody(body: unknown): this {
    this.body = body;
    return this;
  }
  
  setTimeout(timeout: number): this {
    this.timeout = timeout;
    return this;
  }
  
  setRetries(retries: number): this {
    this.retries = retries;
    return this;
  }
  
  enableCache(): this {
    this.cache = true;
    return this;
  }
  
  setAuthToken(token: string): this {
    this.authToken = token;
    this.headers["Authorization"] = `Bearer ${token}`;
    return this;
  }
  
  build(): HttpRequest {
    if (!this.url) {
      throw new Error("URL is required");
    }
    
    return new HttpRequest(
      this.method,
      this.url,
      { ...this.headers },
      this.body,
      this.timeout,
      this.retries,
      this.cache,
      this.authToken
    );
  }
}

// Director: Predefined configurations
class RequestDirector {
  constructor(private builder: RequestBuilder) {}
  
  constructGetRequest(url: string): HttpRequest {
    return this.builder
      .setMethod("GET")
      .setUrl(url)
      .setTimeout(3000)
      .build();
  }
  
  constructPostRequest(url: string, body: unknown): HttpRequest {
    return this.builder
      .setMethod("POST")
      .setUrl(url)
      .addHeader("Content-Type", "application/json")
      .setBody(body)
      .setTimeout(10000)
      .setRetries(3)
      .build();
  }
  
  constructAuthenticatedRequest(url: string, token: string): HttpRequest {
    return this.builder
      .setMethod("GET")
      .setUrl(url)
      .setAuthToken(token)
      .enableCache()
      .build();
  }
}

// Usage
console.log("=== Builder Pattern Demo ===\n");

// Manual building
const request1 = new HttpRequestBuilder()
  .setMethod("POST")
  .setUrl("https://api.example.com/users")
  .addHeader("Content-Type", "application/json")
  .setBody({ name: "John", email: "john@example.com" })
  .setTimeout(10000)
  .setRetries(3)
  .build();

console.log("Manual Request:", request1.toString());
console.log("Headers:", request1.headers);

// Using Director
const director = new RequestDirector(new HttpRequestBuilder());

const getRequest = director.constructGetRequest("https://api.example.com/status");
console.log("\nGET Request:", getRequest.toString());

const postRequest = director.constructPostRequest(
  "https://api.example.com/users",
  { name: "Jane" }
);
console.log("POST Request:", postRequest.toString());

const authRequest = director.constructAuthenticatedRequest(
  "https://api.example.com/protected",
  "token123"
);
console.log("Auth Request:", authRequest.toString());
console.log("Auth Header:", authRequest.headers["Authorization"]);
```

**Explanation:**
- **Product** (`HttpRequest`): The complex object being constructed
- **Builder Interface**: Defines steps to build the product
- **Concrete Builder** (`HttpRequestBuilder`): Implements construction steps, returns `this` for chaining
- **Director** (`RequestDirector`): Defines common construction recipes
- **Benefits**: Immutable objects, readable construction code, validation at build time

---

## 31.4 Observer Pattern

The Observer pattern defines a one-to-many dependency between objects. When one object changes state, all its dependents are notified and updated automatically.

### Implementation

```typescript
// Observer interface
interface Observer<T> {
  update(data: T): void;
}

// Subject interface
interface Subject<T> {
  attach(observer: Observer<T>): void;
  detach(observer: Observer<T>): void;
  notify(data: T): void;
}

// Concrete Subject: Weather Station
class WeatherStation implements Subject<WeatherData> {
  private observers: Set<Observer<WeatherData>> = new Set();
  private currentData: WeatherData | null = null;
  
  attach(observer: Observer<WeatherData>): void {
    this.observers.add(observer);
    console.log("Observer attached");
    
    // Immediately notify with current data if available
    if (this.currentData) {
      observer.update(this.currentData);
    }
  }
  
  detach(observer: Observer<WeatherData>): void {
    this.observers.delete(observer);
    console.log("Observer detached");
  }
  
  notify(data: WeatherData): void {
    this.currentData = data;
    this.observers.forEach(observer => observer.update(data));
  }
  
  // Business logic
  setMeasurements(temperature: number, humidity: number, pressure: number): void {
    const data: WeatherData = {
      temperature,
      humidity,
      pressure,
      timestamp: new Date()
    };
    this.notify(data);
  }
}

interface WeatherData {
  temperature: number;
  humidity: number;
  pressure: number;
  timestamp: Date;
}

// Concrete Observers
class CurrentConditionsDisplay implements Observer<WeatherData> {
  constructor(private station: WeatherStation) {
    this.station.attach(this);
  }
  
  update(data: WeatherData): void {
    console.log(`
      Current Conditions:
      Temperature: ${data.temperature}°F
      Humidity: ${data.humidity}%
      Time: ${data.timestamp.toLocaleTimeString()}
    `);
  }
  
  unsubscribe(): void {
    this.station.detach(this);
  }
}

class StatisticsDisplay implements Observer<WeatherData> {
  private temperatures: number[] = [];
  
  constructor(private station: WeatherStation) {
    this.station.attach(this);
  }
  
  update(data: WeatherData): void {
    this.temperatures.push(data.temperature);
    
    const avg = this.temperatures.reduce((a, b) => a + b, 0) / this.temperatures.length;
    const max = Math.max(...this.temperatures);
    const min = Math.min(...this.temperatures);
    
    console.log(`
      Statistics:
      Avg: ${avg.toFixed(1)}°F
      Max: ${max}°F
      Min: ${min}°F
      Readings: ${this.temperatures.length}
    `);
  }
}

// Usage
const weatherStation = new WeatherStation();

const currentDisplay = new CurrentConditionsDisplay(weatherStation);
const statsDisplay = new StatisticsDisplay(weatherStation);

weatherStation.setMeasurements(80, 65, 30.4);
weatherStation.setMeasurements(82, 70, 29.2);
weatherStation.setMeasurements(78, 90, 29.2);

currentDisplay.unsubscribe();
```

---

## 31.5 Strategy Pattern

The Strategy pattern defines a family of algorithms, encapsulates each one, and makes them interchangeable. It lets the algorithm vary independently from clients that use it.

### Implementation

```typescript
// Strategy interface
interface PaymentStrategy {
  pay(amount: number): Promise<PaymentResult>;
  validate(): boolean;
  getName(): string;
}

interface PaymentResult {
  success: boolean;
  transactionId?: string;
  message: string;
  timestamp: Date;
}

// Concrete Strategy: Credit Card
class CreditCardStrategy implements PaymentStrategy {
  constructor(
    private cardNumber: string,
    private cvv: string,
    private expiryDate: string,
    private cardHolder: string
  ) {}
  
  validate(): boolean {
    // Luhn algorithm validation
    const clean = this.cardNumber.replace(/\s/g, "");
    if (!/^\d{13,19}$/.test(clean)) return false;
    
    let sum = 0;
    let isEven = false;
    
    for (let i = clean.length - 1; i >= 0; i--) {
      let digit = parseInt(clean.charAt(i), 10);
      
      if (isEven) {
        digit *= 2;
        if (digit > 9) digit -= 9;
      }
      
      sum += digit;
      isEven = !isEven;
    }
    
    return sum % 10 === 0;
  }
  
  async pay(amount: number): Promise<PaymentResult> {
    if (!this.validate()) {
      return {
        success: false,
        message: "Invalid credit card",
        timestamp: new Date()
      };
    }
    
    // Simulate API call
    await new Promise(resolve => setTimeout(resolve, 1000));
    
    return {
      success: true,
      transactionId: `CC-${Date.now()}`,
      message: `Paid $${amount} via Credit Card`,
      timestamp: new Date()
    };
  }
  
  getName(): string {
    return "Credit Card";
  }
}

// Concrete Strategy: PayPal
class PayPalStrategy implements PaymentStrategy {
  constructor(
    private email: string,
    private password: string
  ) {}
  
  validate(): boolean {
    return /^[^\s@]+@[^\s@]+\.[^\s@]+$/.test(this.email) && 
           this.password.length >= 8;
  }
  
  async pay(amount: number): Promise<PaymentResult> {
    if (!this.validate()) {
      return {
        success: false,
        message: "Invalid PayPal credentials",
        timestamp: new Date()
      };
    }
    
    await new Promise(resolve => setTimeout(resolve, 800));
    
    return {
      success: true,
      transactionId: `PP-${Date.now()}`,
      message: `Paid $${amount} via PayPal`,
      timestamp: new Date()
    };
  }
  
  getName(): string {
    return "PayPal";
  }
}

// Context: Shopping Cart
class ShoppingCart {
  private items: { name: string; price: number }[] = [];
  private paymentStrategy: PaymentStrategy | null = null;
  
  addItem(name: string, price: number): void {
    this.items.push({ name, price });
  }
  
  getTotal(): number {
    return this.items.reduce((sum, item) => sum + item.price, 0);
  }
  
  setPaymentStrategy(strategy: PaymentStrategy): void {
    this.paymentStrategy = strategy;
  }
  
  async checkout(): Promise<PaymentResult> {
    if (!this.paymentStrategy) {
      return {
        success: false,
        message: "No payment method selected",
        timestamp: new Date()
      };
    }
    
    const total = this.getTotal();
    console.log(`\nChecking out ${this.items.length} items, total: $${total}`);
    console.log(`Using: ${this.paymentStrategy.getName()}`);
    
    return this.paymentStrategy.pay(total);
  }
}

// Usage
async function main() {
  const cart = new ShoppingCart();
  cart.addItem("Laptop", 999.99);
  cart.addItem("Mouse", 29.99);
  
  // Pay with credit card
  cart.setPaymentStrategy(
    new CreditCardStrategy("4532015112830366", "123", "12/25", "John Doe")
  );
  const result1 = await cart.checkout();
  console.log(result1);
}

main();
```

---

## 31.6 Command Pattern

The Command pattern encapsulates a request as an object, thereby letting you parameterize clients with different requests, queue or log requests, and support undoable operations.

### Implementation

```typescript
// Command interface
interface Command {
  execute(): void;
  undo(): void;
  getDescription(): string;
}

// Receiver
class TextEditor {
  private content = "";
  private clipboard = "";
  
  getContent(): string {
    return this.content;
  }
  
  setContent(content: string): void {
    this.content = content;
  }
  
  getClipboard(): string {
    return this.clipboard;
  }
  
  setClipboard(text: string): void {
    this.clipboard = text;
  }
  
  insertText(position: number, text: string): void {
    this.content = this.content.slice(0, position) + text + this.content.slice(position);
  }
  
  deleteText(start: number, end: number): string {
    const deleted = this.content.slice(start, end);
    this.content = this.content.slice(0, start) + this.content.slice(end);
    return deleted;
  }
}

// Concrete Commands
class WriteCommand implements Command {
  private previousContent: string;
  
  constructor(
    private editor: TextEditor,
    private text: string
  ) {
    this.previousContent = editor.getContent();
  }
  
  execute(): void {
    this.editor.setContent(this.previousContent + this.text);
  }
  
  undo(): void {
    this.editor.setContent(this.previousContent);
  }
  
  getDescription(): string {
    return `Write "${this.text}"`;
  }
}

class DeleteCommand implements Command {
  private deletedText: string;
  
  constructor(
    private editor: TextEditor,
    private start: number,
    private end: number
  ) {}
  
  execute(): void {
    this.deletedText = this.editor.deleteText(this.start, this.end);
  }
  
  undo(): void {
    this.editor.insertText(this.start, this.deletedText);
  }
  
  getDescription(): string {
    return `Delete from ${this.start} to ${this.end}`;
  }
}

// Invoker
class CommandManager {
  private history: Command[] = [];
  private redoStack: Command[] = [];
  
  execute(command: Command): void {
    command.execute();
    this.history.push(command);
    this.redoStack = [];
    console.log(`Executed: ${command.getDescription()}`);
  }
  
  undo(): void {
    const command = this.history.pop();
    if (command) {
      command.undo();
      this.redoStack.push(command);
      console.log(`Undone: ${command.getDescription()}`);
    } else {
      console.log("Nothing to undo");
    }
  }
  
  redo(): void {
    const command = this.redoStack.pop();
    if (command) {
      command.execute();
      this.history.push(command);
      console.log(`Redone: ${command.getDescription()}`);
    } else {
      console.log("Nothing to redo");
    }
  }
  
  getHistory(): string[] {
    return this.history.map(cmd => cmd.getDescription());
  }
}

// Usage
const editor = new TextEditor();
const commandManager = new CommandManager();

console.log("=== Text Editor Demo ===\n");

commandManager.execute(new WriteCommand(editor, "Hello World"));
console.log("Content:", editor.getContent());

commandManager.execute(new DeleteCommand(editor, 5, 11));
console.log("Content:", editor.getContent());

console.log("\n--- Undoing ---");
commandManager.undo();
console.log("Content:", editor.getContent());
```

---

## 31.7 Decorator Pattern

The Decorator pattern allows behavior to be added to individual objects, either statically or dynamically, without affecting the behavior of other objects from the same class.

### Implementation

```typescript
// Component interface
interface Coffee {
  cost(): number;
  description(): string;
  ingredients(): string[];
}

// Concrete Component
class SimpleCoffee implements Coffee {
  cost(): number {
    return 10;
  }
  
  description(): string {
    return "Simple coffee";
  }
  
  ingredients(): string[] {
    return ["coffee", "water"];
  }
}

// Base Decorator
abstract class CoffeeDecorator implements Coffee {
  protected decoratedCoffee: Coffee;
  
  constructor(coffee: Coffee) {
    this.decoratedCoffee = coffee;
  }
  
  cost(): number {
    return this.decoratedCoffee.cost();
  }
  
  description(): string {
    return this.decoratedCoffee.description();
  }
  
  ingredients(): string[] {
    return this.decoratedCoffee.ingredients();
  }
}

// Concrete Decorators
class MilkDecorator extends CoffeeDecorator {
  cost(): number {
    return super.cost() + 2;
  }
  
  description(): string {
    return `${super.description()}, milk`;
  }
  
  ingredients(): string[] {
    return [...super.ingredients(), "milk"];
  }
}

class SugarDecorator extends CoffeeDecorator {
  private sugarLevel: number;
  
  constructor(coffee: Coffee, sugarLevel: number = 1) {
    super(coffee);
    this.sugarLevel = sugarLevel;
  }
  
  cost(): number {
    return super.cost() + (0.5 * this.sugarLevel);
  }
  
  description(): string {
    return `${super.description()}, ${this.sugarLevel}x sugar`;
  }
  
  ingredients(): string[] {
    return [...super.ingredients(), `sugar (${this.sugarLevel})`];
  }
}

class WhipDecorator extends CoffeeDecorator {
  cost(): number {
    return super.cost() + 3;
  }
  
  description(): string {
    return `${super.description()}, whipped cream`;
  }
  
  ingredients(): string[] {
    return [...super.ingredients(), "whipped cream"];
  }
}

// Usage
let coffee: Coffee = new SimpleCoffee();
coffee = new MilkDecorator(coffee);
coffee = new SugarDecorator(coffee, 2);
coffee = new WhipDecorator(coffee);

console.log(coffee.description());
console.log(coffee.cost());
console.log(coffee.ingredients());
```

---

## 31.8 Proxy Pattern

The Proxy pattern provides a surrogate or placeholder for another object to control access to it.

### Implementation

```typescript
// Subject interface
interface UserService {
  getUser(id: number): Promise<User>;
  saveUser(user: User): Promise<void>;
  deleteUser(id: number): Promise<void>;
  listUsers(): Promise<User[]>;
}

interface User {
  id: number;
  name: string;
  email: string;
  role: "admin" | "user" | "guest";
}

// Real Subject
class RealUserService implements UserService {
  private users = new Map<number, User>();
  private nextId = 1;
  
  async getUser(id: number): Promise<User> {
    await this.simulateNetwork();
    const user = this.users.get(id);
    if (!user) throw new Error(`User ${id} not found`);
    return user;
  }
  
  async saveUser(user: User): Promise<void> {
    await this.simulateNetwork();
    if (user.id === 0) user.id = this.nextId++;
    this.users.set(user.id, user);
  }
  
  async deleteUser(id: number): Promise<void> {
    await this.simulateNetwork();
    this.users.delete(id);
  }
  
  async listUsers(): Promise<User[]> {
    await this.simulateNetwork();
    return Array.from(this.users.values());
  }
  
  private simulateNetwork(): Promise<void> {
    return new Promise(resolve => setTimeout(resolve, 100));
  }
}

// Proxy: Logging Proxy
class LoggingProxy implements UserService {
  constructor(private service: UserService) {}
  
  async getUser(id: number): Promise<User> {
    console.log(`[LOG] Getting user ${id}`);
    const start = Date.now();
    try {
      const user = await this.service.getUser(id);
      console.log(`[LOG] Got user ${id} in ${Date.now() - start}ms`);
      return user;
    } catch (error) {
      console.error(`[LOG] Failed to get user ${id}: ${error}`);
      throw error;
    }
  }
  
  async saveUser(user: User): Promise<void> {
    console.log(`[LOG] Saving user ${user.id || "new"}`);
    return this.service.saveUser(user);
  }
  
  async deleteUser(id: number): Promise<void> {
    console.log(`[LOG] Deleting user ${id}`);
    return this.service.deleteUser(id);
  }
  
  async listUsers(): Promise<User[]> {
    console.log(`[LOG] Listing users`);
    return this.service.listUsers();
  }
}

// Proxy: Caching Proxy
class CachingProxy implements UserService {
  private cache = new Map<string, { data: unknown; timestamp: number }>();
  private ttl: number;
  
  constructor(private service: UserService, ttlSeconds: number = 60) {
    this.ttl = ttlSeconds * 1000;
  }
  
  async getUser(id: number): Promise<User> {
    const cacheKey = `user_${id}`;
    const cached = this.getFromCache<User>(cacheKey);
    
    if (cached) {
      console.log(`[CACHE] Hit for user ${id}`);
      return cached;
    }
    
    console.log(`[CACHE] Miss for user ${id}`);
    const user = await this.service.getUser(id);
    this.setCache(cacheKey, user);
    return user;
  }
  
  async saveUser(user: User): Promise<void> {
    await this.service.saveUser(user);
    this.cache.delete(`user_${user.id}`);
    this.cache.delete("users_list");
  }
  
  async deleteUser(id: number): Promise<void> {
    await this.service.deleteUser(id);
    this.cache.delete(`user_${id}`);
    this.cache.delete("users_list");
  }
  
  async listUsers(): Promise<User[]> {
    const cacheKey = "users_list";
    const cached = this.getFromCache<User[]>(cacheKey);
    
    if (cached) {
      console.log(`[CACHE] Hit for user list`);
      return cached;
    }
    
    console.log(`[CACHE] Miss for user list`);
    const users = await this.service.listUsers();
    this.setCache(cacheKey, users);
    return users;
  }
  
  private getFromCache<T>(key: string): T | null {
    const entry = this.cache.get(key);
    if (!entry) return null;
    
    if (Date.now() - entry.timestamp > this.ttl) {
      this.cache.delete(key);
      return null;
    }
    
    return entry.data as T;
  }
  
  private setCache(key: string, data: unknown): void {
    this.cache.set(key, { data, timestamp: Date.now() });
  }
  
  clearCache(): void {
    this.cache.clear();
    console.log("[CACHE] Cleared");
  }
}

// Proxy: Access Control Proxy
class AuthorizationProxy implements UserService {
  constructor(
    private service: UserService,
    private currentUserRole: "admin" | "user" | "guest"
  ) {}
  
  async getUser(id: number): Promise<User> {
    return this.service.getUser(id);
  }
  
  async saveUser(user: User): Promise<void> {
    this.checkPermission("admin", "save users");
    return this.service.saveUser(user);
  }
  
  async deleteUser(id: number): Promise<void> {
    this.checkPermission("admin", "delete users");
    return this.service.deleteUser(id);
  }
  
  async listUsers(): Promise<User[]> {
    this.checkPermission("user", "list users");
    return this.service.listUsers();
  }
  
  private checkPermission(required: "admin" | "user" | "guest", action: string): void {
    const hierarchy = { guest: 0, user: 1, admin: 2 };
    if (hierarchy[this.currentUserRole] < hierarchy[required]) {
      throw new Error(`Access denied: ${this.currentUserRole} cannot ${action}`);
    }
  }
}

// Usage: Composing proxies
async function demonstrateProxies() {
  console.log("=== Proxy Pattern Demo ===\n");
  
  const realService = new RealUserService();
  const loggedService = new LoggingProxy(realService);
  const cachedService = new CachingProxy(loggedService, 5);
  const authorizedService = new AuthorizationProxy(cachedService, "admin");
  
  console.log("--- Creating User ---");
  await authorizedService.saveUser({
    id: 0,
    name: "John Doe",
    email: "john@example.com",
    role: "user"
  });
  
  console.log("\n--- Fetching User (Cache Miss) ---");
  await authorizedService.getUser(1);
  
  console.log("\n--- Fetching User (Cache Hit) ---");
  await authorizedService.getUser(1);
  
  console.log("\n--- Testing Authorization ---");
  const guestService = new AuthorizationProxy(realService, "guest");
  try {
    await guestService.deleteUser(1);
  } catch (error) {
    console.error(error);
  }
}

demonstrateProxies();
```

---

## 31.8 Chapter Summary and Exercises

### Chapter Summary

This chapter covered essential design patterns that solve common architectural challenges:

**Key Patterns:**

1. **Singleton Pattern**: Ensures single instance existence, useful for shared resources like database connections or configuration managers.

2. **Factory Pattern**: Encapsulates object creation, allowing code to work with interfaces rather than concrete classes. Includes Simple Factory and Abstract Factory variations.

3. **Builder Pattern**: Separates complex object construction from representation, providing fluent APIs for step-by-step construction with many optional parameters.

4. **Observer Pattern**: Establishes subscription mechanism to notify multiple objects about events, fundamental for event-driven architectures.

5. **Strategy Pattern**: Encapsulates interchangeable algorithms, allowing runtime selection of behaviors (e.g., different payment methods).

6. **Command Pattern**: Encapsulates requests as objects, enabling undo/redo functionality, queuing, and logging of operations.

7. **Decorator Pattern**: Adds responsibilities to objects dynamically, providing flexible alternatives to subclassing for extending functionality.

8. **Proxy Pattern**: Provides placeholders for objects to control access, add logging, caching, or authorization without modifying the original object.

### Practical Exercises

**Exercise 1: Singleton Pattern**

Create a configuration manager singleton that:
- Loads configuration from environment variables or JSON files
- Provides type-safe access to config values
- Supports hot-reloading in development mode
- Prevents multiple instances

**Exercise 2: Factory Pattern**

Implement a cross-platform UI component factory:
- Create buttons, text inputs, and checkboxes
- Support Material Design and Bootstrap themes
- Add a new theme (e.g., Tailwind) without modifying existing code
- Use the factory in a form builder

**Exercise 3: Builder Pattern**

Build a SQL query builder:
- Support SELECT, INSERT, UPDATE, DELETE operations
- Allow chaining methods (fluent interface)
- Support WHERE clauses, JOINs, ORDER BY, LIMIT
- Generate parameterized queries to prevent SQL injection
- Validate the query before building

**Exercise 4: Observer Pattern**

Create a stock price monitoring system:
- Stock class as subject that updates prices randomly
- Multiple observers: PriceAlert (sends notification when threshold reached), PriceLogger (logs to file), PortfolioUpdater (updates portfolio value)
- Support adding/removing observers dynamically
- Ensure type safety for different stock types

**Exercise 5: Strategy Pattern**

Implement a sorting algorithm selector:
- Define strategies: QuickSort, MergeSort, BubbleSort, HeapSort
- Each strategy implements the same interface with methods: sort(array), getComplexity(), getName()
- Context class receives large datasets and selects appropriate algorithm based on data size
- Benchmark different strategies

**Exercise 6: Command Pattern**

Build a drawing application command system:
- Commands: DrawLine, DrawCircle, DrawRectangle, FillColor, ClearCanvas
- Each command stores parameters and can execute/undo
- Command manager with history panel showing all executed commands
- Support macro commands (group multiple commands)
- Save/load command history

**Exercise 7: Decorator Pattern**

Create a data compression and encryption pipeline:
- Base component: FileDataSource that reads/writes files
- Decorators: CompressionDecorator (compresses data), EncryptionDecorator (encrypts data), BufferingDecorator (buffers I/O)
- Combine decorators: encrypted + compressed file storage
- Ensure decorators can be stacked in any order
- Measure performance impact

**Exercise 8: Proxy Pattern**

Implement a smart image loading proxy:
- RealSubject: HighResolutionImage that loads large images from disk
- Proxy: ImageProxy that displays placeholder while loading, caches loaded images, implements lazy loading
- Virtual Proxy: Create proxy for expensive objects that delays creation until needed
- Protection Proxy: Add access control to image operations based on user permissions
- Logging Proxy: Log all image access for analytics

### Additional Resources

- **"Design Patterns: Elements of Reusable Object-Oriented Software"** by Gang of Four (GoF)
- **Refactoring Guru - Design Patterns**: https://refactoring.guru/design-patterns
- **TypeScript Design Patterns**: https://github.com/torokmark/design_patterns_in_typescript
- **Head First Design Patterns**: Excellent visual explanations of patterns

---

## Coming Up Next: Chapter 32 - TypeScript with React

In the next chapter, we will explore integrating TypeScript with React, the most popular frontend library. We'll cover:

- Setting up React projects with TypeScript
- Typing component props and state
- Functional components and Hooks (useState, useEffect, useContext)
- Event handling with proper types
- Creating type-safe custom hooks
- Higher-Order Components (HOCs) with TypeScript
- Render props pattern implementation
- Context API with TypeScript
- Generic components for reusable UI elements

Understanding TypeScript with React is essential for building type-safe, maintainable frontend applications with excellent developer experience and fewer runtime errors.